Conectarse al servidor asi:<br>
">" mongod -port 28000 -dbpath C:\data<br>
Luego:<br>
">" mongo twitter -port 28000

In [21]:
from pymongo import MongoClient
client = MongoClient('mongodb://localhost:28000/')

In [22]:
db = client['twitter']

In [23]:
tweets = db['tweets']

In [24]:
tweet = {
    '_id': 2,
    'usuario': {'nick': "herminia", 'seguidores':5320},
    'texto': "RT:@herminia: Hoy, excursion ala sierra con @aniceto!",
    'menciones': ["hermiia", "aniceto"],
    'RT': True,
    'origen': 1
}

In [25]:
insertado = tweets.insert_one(tweet)

In [26]:
print(insertado.inserted_id)

2


<h3>Importacion de ficheros</h3>

In [39]:
from pymongo import MongoClient
import random
import string
client = MongoClient('mongodb://localhost:28000/')
db = client['twitter']
tweets = db['tweets']
tweets.drop()

Usuarios inventados

In [40]:
usuarios = [("bertoldo", 1320),("herminia", 5320),("aniceto", 123),("melibea", 411)]
n = 100

In [41]:
for i in range(1,n+1):
    tweet = {}
    tweet['_id'] = i
    tweet['text'] = ''.join(random.choices(string.ascii_uppercase, k=10))
    u = {}
    u['nick'], u['seguidores'] = random.choice(usuarios)
    tweet['usuario'] = u
    tweet['RT'] = i>1 and random.choice([False,True])
    if tweet['RT'] and i>1:
        tweet['origen'] = random.randrange(1,i)
    m = random.sample(usuarios, random.randrange(0, len(usuarios)))
    tweet['mentions'] = [nick for nick,_ in m]
    #tweets.insert_one(tweet)
    insertado = tweets.insert_one(tweet)

In [42]:
print(insertado.inserted_id)

100


">"db.tweets.count()

<H3>Consultas simples</H3>

<p>Modo simple, busca 20 tweets:
">" db.tweets.find()<br>
Mejorar la vista del documento:
">"db.tweets.find().pretty()<br>
Empezar desde el segundo documento:
">"db.tweets.find().skip(1).pretty()<br> 
Mostrar los n primeros documentos con limit(n):
">"db.tweets.find().skip(5).limit(2).pretty()<br> 
Ordenar documentos con sort:<br>
+1 (Menor a Mayor), -1(Mayor a Menor)<br>
">"db.tweets.find().skip(5).sort({_id:-1}).pretty()<br>
Ordenar por numero de seguidores, de Mayor a Menor:
">"db.tweets.find().sort({"usuario.seguidores":-1}).pretty()<br>
Ordenar por numero de seguidores, de Mayor a Menor, tambien para los tweets del usuario con el mismo tweet:
">"db.tweets.find().sort({"usuario.seguidores":-1, _id:-1}).pretty()<br>
Combinar limit, skip y sort:
">"db.tweets.find().sort({_id:-1}).limit(1)<br>
Pero Tambien:
">"db.tweets.find().limit(1).sort({_id:-1})<br>
Crear un index para acelerar la busqueda:
">"db.tweets.createIndex({"usuario.seguidores":-1, _id:-1})<br></p>

<h3>Estructura general de find</h3>

find({filtro},{proyeccion})<br>
Para proyeccion:<br>
find({},{clave:1}): incluya la clave<br>
find({},{clave:0}): excluya la clave<br>
<p>Excluye el usuario</p>
<p>>db.tweets.find({},{usuario:0})</p>
<p>Solo ver id y text, si no se pone ID igual lo mostrara</p>
<p>>db.tweets.find({},{_id:1, text:1})</p>
<p>Solo ver text</p>
<p>>db.tweets.find({},{text:1, _id:0})</p>

<h3>Igualdad</h3>

<p>Solo ver textos del retweets</p>
<p>>db.tweets.find({RT:true},{text:1, _id:0})</p>
<p>Solo ver retweets hechos por Bertoldo</p>
<p>>db.tweets.find({RT:true, 'usuario.nick':'bertoldo'},{text:1, _id:0})</p>
<p>contar retweets hechos por Bertoldo</p>
<p>>db.tweets.find({RT:true, 'usuario.nick':'bertoldo'},{text:1, _id:0}).count()</p>

<h3>Operadores de comparacion</h3>

<p>tweets no emitidos por Bertoldo</p><br>
<p>>db.tweets.find({'usuario.nick':{&dollar;ne:'bertoldo'}}).count()</p><br>
<p>Restar al total el numero de tweets emitidos por Bertoldo</p><br>
<p>>db.tweets.find().count() - db.tweets.find({'usuario.nick':'bertoldo'}).count()</p><br>
<p>Tweets de usuarios que tienen entre 1000 y 2000</p><br>
<p>>db.tweets.find({'usuario.seguidores':{&dollar;gt:1000, &dollar;lt:2000}})</p><br>
<p>Tweets Escritos por Bertoldo o Herminia</p><br>
<p>>db.tweets.find({'$or': [{'usuario.nick':'bertoldo'},{'usuario.nick':'herminia'}]})</p>

<h3>ARRAYS</h3>

<p>Ver la clave mentions</p>
<p>db.tweets.find({mentions:"aniceto"},{mentions:1})</p>
<p>No menciona aniceto</p>
<p>db.tweets.find({mentions:{&dollar;ne:"aniceto"}},{mentions:1})</p>

<h3>$EXISTS</h3>

<p>Mostrar Tweets por clave original</p>
<p>>db.tweets.find().sort({origen:1})</p>
<p>Mostrar Tweets por clave original cuando una clave no existe</p>
<p>>db.tweets.find({origen:{&dollar;exists:1}}).sort({origen:1})</p>

<h3>Find in Python</h3>

In [1]:
from pymongo import MongoClient
client = MongoClient('mongodb://localhost:28000/')
db = client['twitter']
tweets = db['tweets']

In [2]:
tweet = tweets.find_one({"usuario.nick": 'bertoldo'}, {'text':1, '_id':0})
print(tweet)

{'text': 'GZKPZTBQUH'}


In [3]:
tweet = tweets.find_one({"usuario.nick": 'bertoldo'})
print('text: ', tweet['text'])

text:  GZKPZTBQUH


In [4]:
for t in tweets.find({"usuario.nick": 'bertoldo', 'mentions': "herminia"}):
    print(t['text'])

GZKPZTBQUH
ITZAJNFTAS
FBQZLVIEVT
MDTECYLPWW
CZOCEBPXNM
AYAAUOLIQG
XDVGSLEFJT
ASBQGRQAZE
RKKMQSLRHL
NEKKAEFVIS
RSRZQCTOWM
UMIAJLQIAG


In [6]:
for t in tweets.find():
    if t['usuario']['nick'] == "bertoldo" and "herminia" in t['mentions']:
        print(t['text'])

GZKPZTBQUH
ITZAJNFTAS
FBQZLVIEVT
MDTECYLPWW
CZOCEBPXNM
AYAAUOLIQG
XDVGSLEFJT
ASBQGRQAZE
RKKMQSLRHL
NEKKAEFVIS
RSRZQCTOWM
UMIAJLQIAG


<h2>Agregaciones</h2>
<h3>El piepline</h3>
<p>aggregate se define mediante una serie de etapas consecutivas</p>
<p>>db.tweets.aggregate([etapa1, ..., etapan])</p>

<h3>&dollar;group</h3>
<p>Agrupar elementos y realizar operaciones sobre cada grupo</p>
<p>>db.tweets.aggregate(
     [
      {&dollar;group:
        {_id:"&dollar;usuario.nick",
        num_tweets:{&dollar;sum:1}
        }
      }
     ]
    )</p><br>
<p>>db.tweets.aggregate([{$sortByCount:"$usuario.nick"}])</p>

<p>>db.tweets.aggregate( [ {$group: {_id:{nick:"$usuario.nick", RT:"$RT"}, num_tweets:{$sum:1} } } ] )
</p>

<p>>db.tweets.aggregate( [ {$group: {_id:"$usuario.nick", textos:{$push:"$text"} } } ] )</p>

<p>db.tweets.aggregate( [ {$group: {_id:null, menciones:{$avg:{$size: "$mentions"}} } } ] )</p>

<h2>$match</h2>

<p>Ver el total de tweets por usuario con mas de 20 tweets</p>
<p>db.tweets.aggregate( [ {$sortByCount: "$usuario.nick"}, {$match: {count:{$gt:20}}} ] )</p>

<h3>$project</h3>
<p>Crear el campo nuMentions que contendra el tamaño del campo mentios</p>
<p>db.tweets.aggregate( [ {$project: {usuario:1, _id:0, numMentions : {$size:"$mentions"}}}] )</p>

<h3>$unwind, $sample, $out,..</h3>

<p>db.unwind.drop()</p>
<p>db.unwind.insert({_id:1, a:1, b:[1,2,3]})</p>
<p>db.unwind.insert({_id:2, a:2, b:[4,5]})</p>
<p>db.unwind.aggregate([{$unwind: "$b"}])</p>

Combinado con otros:
db.tweets.aggregate([{$sample:{size: 2}}])

$out: almacena el resultado de las etapas anteriores <br>
db.tweets.aggregate([{$sample:{size: 3}},{$out: "minitweets"}])

<h3>$lookup</h3>

db.tweets.findOne({RT: true})<br>
Estructura Lookup<br>
{
    $lookup:
    {
        from:<Coleccion a combinar>,
        localField:<Clave de los documentos riginales>,
        foreignField:<Clave de los documentos de la coleccion "From">,
        as:<nombre del campo del array generado>
    }
}

db.tweets.aggregate([{$match:{RT: true}},{$lookup:{from: "tweets",localField: "origen",foreignField: "_id",as: "tweet_original"}}, {$unwind:"$tweet_original"},{$project: {_id: "$_id", emitido: "$usuario.nick", fuente:"$tweet_original.usuario.nick"}}])

Usuario mas mencionado:<br>
db.tweets.aggregate([{$match:{RT: true}},{$unwind:"$mentions"},{$sortByCount: "$mentions"}])

<h2>Vistas</h2>

db.createView("mencionesOriginales", "tweets", [{$match:{RT: true}},{$unwind:"$mentions"},{$sortByCount: "$mentions"}])

<p>>show collections</p>
<p>>db.mencionesOriginales.find()</p>

<h2>Update and remove</h2>

In [1]:
from pymongo import MongoClient
client = MongoClient('mongodb://localhost:28000/')
db = client['astronomia']
estelar = db['estelar']

In [2]:
estelar.drop()

In [4]:
estelar.insert_many([
    {'_id':1, 'nombre': "Sirio", 'tipo': "estrella", 'espectro': "A1V"},
    {'_id':2, 'nombre': "Saturno", 'tipo': "Planeta"},
    {'_id':3, 'nombre': "Pluton", 'tipo': "Planeta"}
])

In [5]:
pluton = estelar.find_one({'_id':3})
pluton['tipo'] = "planeta enano"
estelar.replace_one({'_id':pluton['_id']}, pluton)

<h3>Update Parcial</h3>

<p>>use astronomia</p>

db.estelar.updateOne({nombre:"Pluton"}, {$set: {tipo: "planeta enano"}})

Para modificar varios valores: <br>
db.estelar.updateMany({}, {$currentDate:{fecha: true}})

Clave "Tipo" canmbie a "Clase": <br>
db.estelar.updateMany({}, {$rename:{"tipo": "clase"}})

Eliminar la clave "espectro": <br>
db.estelar.updateOne({nombre:"Sirio"}, {$unset:{"espectro": true}})

Eliminar elementos:<br>
db.estelar.remove({clase:'planeta enano'})<br>
db.estelar.remove({clase:'planeta'}, {justOne: true})